## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,52.09,75,98,11.59,overcast clouds
1,1,Miyang,CN,24.4042,103.4428,63.48,99,100,2.93,overcast clouds
2,2,Upernavik,GL,72.7868,-56.1549,42.93,80,100,10.71,overcast clouds
3,3,Albany,US,42.6001,-73.9662,86.85,71,71,2.91,broken clouds
4,4,Longyearbyen,SJ,78.2186,15.6401,48.04,81,75,2.30,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 76
What is the maximum temperature you would like for your trip? 89


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Albany,US,42.6001,-73.9662,86.85,71,71,2.91,broken clouds
7,7,Almeirim,BR,-1.5233,-52.5817,79.07,79,95,2.62,overcast clouds
10,10,Iquitos,PE,-3.7481,-73.2472,84.24,70,40,6.91,scattered clouds
13,13,Jamestown,US,42.0970,-79.2353,83.79,61,40,12.66,scattered clouds
15,15,Hede,CN,33.7681,120.2517,76.05,96,100,17.34,moderate rain
16,16,Georgetown,MY,5.4112,100.3354,80.89,88,20,1.01,few clouds
20,20,Kapaa,US,22.0752,-159.3190,84.52,76,75,4.00,broken clouds
22,22,Alofi,NU,-19.0595,-169.9187,78.69,83,40,18.41,scattered clouds
24,24,San Cristobal,VE,7.7669,-72.2250,82.58,55,41,5.50,scattered clouds
29,29,Sao Felix Do Xingu,BR,-6.6447,-51.9950,81.30,42,36,1.03,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                165
City                   165
Country                165
Lat                    165
Lng                    165
Max Temp               165
Humidity               165
Cloudiness             165
Wind Speed             165
Current Description    165
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Albany,US,86.85,broken clouds,42.6001,-73.9662,
7,Almeirim,BR,79.07,overcast clouds,-1.5233,-52.5817,
10,Iquitos,PE,84.24,scattered clouds,-3.7481,-73.2472,
13,Jamestown,US,83.79,scattered clouds,42.0970,-79.2353,
15,Hede,CN,76.05,moderate rain,33.7681,120.2517,
16,Georgetown,MY,80.89,few clouds,5.4112,100.3354,
20,Kapaa,US,84.52,broken clouds,22.0752,-159.3190,
22,Alofi,NU,78.69,scattered clouds,-19.0595,-169.9187,
24,San Cristobal,VE,82.58,scattered clouds,7.7669,-72.2250,
29,Sao Felix Do Xingu,BR,81.30,scattered clouds,-6.6447,-51.9950,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        clean_hotel_df["Hotel Name"].replace('', np.nan, inplace=True)     

In [8]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Almeirim,BR,79.07,overcast clouds,-1.5233,-52.5817,Ricácio Hotel
10,Iquitos,PE,84.24,scattered clouds,-3.7481,-73.2472,Casa Morey Hotel & Restaurant
13,Jamestown,US,83.79,scattered clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
15,Hede,CN,76.05,moderate rain,33.7681,120.2517,Vienna 3 Best Hotel Sheyang Jiefang Rd
16,Georgetown,MY,80.89,few clouds,5.4112,100.3354,Cititel Penang


In [9]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [11]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))